In [21]:
import pandas as pd
import os
import numpy as np
import glob
from pathlib import Path
from datetime import datetime as dt

In [22]:
home = Path.home()
target = Path (home, 'HP Inc','GPSTW SOP - 2021 日新','Project team','Single shortage')

In [23]:
ODMdict = {
    'FWH' : 'WHFXN',
    'Compal' : 'KSCEI',
    'Wistron' : 'CQWIS',
    'Inventec' : 'CQIEC',
    'Quanta' : 'CQQCI',
    'Pegatron' : 'CQPCQ'
}

In [24]:
fileList = [str(x) for x in target.glob("*xlsx")]

In [25]:
errorList = []

In [26]:
resultList = []

In [33]:

def clean(fname: str, file : pd.DataFrame) -> pd.DataFrame:

    #add report day
    currentYear = dt.now().year
    #print(fname)
    currentday = fname.split('\\')[-1][-13:-5]
    #currentday = str(currentYear) + currentday
    #print(currentday)
    #print(type(currentday))
    file = file.assign(reportDate = currentday)
    file['reportDate'] = file['reportDate'].apply(lambda x: dt.strptime(x, '%Y%m%d'))
    file['reportDate'] = pd.to_datetime(file['reportDate'])

    #clean
    file.columns = file.columns.str.strip()

    #drop useless columns and rows
    file = file.drop(columns = ['Description (Item)', 'Schedule (Comments)', 'Hub inventory', 'Vendor'])
    file = file[file['Procurement type'] == 'B/S'].reset_index(drop = True)

    #adjust qty columns name and units
    qtycol = file.filter(like='Single Shortage QTY (K)').columns.tolist()
    for i in qtycol:
        file[i] = file[i].replace("New", 0)
        file[i] = file[i].apply(lambda x: x*1000)
    file = file.rename(columns= {qtycol[0]: 'Single Shortage QTY', qtycol[1]: 'Prev_Single Shortage QTY'})

    #replace ODM name
    file['ODM'] = file['ODM'].replace(ODMdict)
    return file    

In [35]:
for f in fileList:
    # file = pd.read_excel(f)
    # resultList.append(clean(f, file))


    try:
        file = pd.read_excel(f)
        resultList.append(clean(f, file))
        print(f + " process done!")
    except Exception as e:
        errorList.append([f, e])
        print(f + " process failed!")

C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Single shortage\single shortage 20211123.xlsx
20211123
<class 'str'>
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Single shortage\single shortage 20211123.xlsx process failed!
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Single shortage\single shortage 20211130.xlsx
20211130
<class 'str'>
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Single shortage\single shortage 20211130.xlsx process failed!
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Single shortage\single shortage 20211202.xlsx
20211202
<class 'str'>
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Single shortage\single shortage 20211202.xlsx process failed!
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Single shortage\single shortage 20211207.xlsx
20211207
<class 'str'>
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\Single shortage\single shortage 20211207.xlsx process failed!
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Pro

In [37]:
result = pd.concat(resultList)

In [40]:
result['Unnamed: 14'].unique()

array([nan], dtype=object)